# fuzzyDist.py
## edge list creation script
## author: J Curry
## date: 09/03/2022

In [1]:
import csv
#from fuzzywuzzy import fuzz
#from fuzzywuzzy import process
import pandas as pd
from tqdm.notebook import tqdm
from alive_progress import alive_bar
import time

from thefuzz import fuzz
from thefuzz import process

import numpy as np

## might not need these??
import random
import gc 
import timeit
import sys
from datetime import datetime as dt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import re



from scipy.sparse import csr_matrix
#%pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

import time

import ftfy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fishc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
refsDf = pd.read_csv('citationNetworks/Data/SplitRefs.csv')
#refsDf['ref_title'] = str(refsDf['ref_title'])
refsDf = refsDf[['X','ID','Title', 'ref_title']]
refsDf.set_axis(['Index', 'ID','Title', 'ref_title'], axis=1)


,Index,ID,Title,ref_title
0,1,4,APHIS: A new software for photo-matching in ec...,A computer-assisted system for photographic m...
1,2,4,APHIS: A new software for photo-matching in ec...,"25 (11), pp. 120-126"
2,3,4,APHIS: A new software for photo-matching in ec...,(2008) An Analysis of Utilizing the Leatherba...
3,4,4,APHIS: A new software for photo-matching in ec...,Application of a method for individual photog...
4,5,4,APHIS: A new software for photo-matching in ec...,Spatial heterogeneity in the effects of clima...
...,...,...,...,...
351098,351281,6372,Methods for wildlife monitoring in tropical fo...,(1996) Measuring and monitoring biological di...
351099,351282,6372,Methods for wildlife monitoring in tropical fo...,Acoustic monitoring for conservation in tropi...
351100,351283,6372,Methods for wildlife monitoring in tropical fo...,Monitoring of biological diversity in space a...
351101,351284,6372,Methods for wildlife monitoring in tropical fo...,Software to facilitate and streamline camera ...


In [11]:
titlesAndIDs = refsDf[['ID','Title']]
titlesAndIDs.set_axis(['ID', 'ref_title'], axis=1)
refsDf = pd.concat([refsDf, titlesAndIDs])
refsDf.reset_index(inplace=True, drop=True)

#print(refsDf)

In [12]:
refsDf['ref_title'] = refsDf['ref_title'].astype(str)

In [13]:
STOPWORDS = stopwords.words('english')
STOPWORDS = set(STOPWORDS)
    
def text_prepare(text, STOPWORDS):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) 
    text = text.strip()
    return text

In [14]:
text_prepare(" 'Which camera trap type and how many do I need?'' A review of camera features and study designs for a range of wildlife research applications (2013) Hystrix, 24, pp. 148-156", STOPWORDS)

'camera trap type many need? review camera features study designs range wildlife research applications 2013 hystrix 24 pp. 148-156'

In [15]:
print(type(refsDf['ref_title']))

<class 'pandas.core.series.Series'>


In [16]:
refsDf['ref_title'] = refsDf['ref_title'].astype(str)
refsDf['ProcessedRef'] = refsDf['ref_title'].apply(lambda x: text_prepare(x, STOPWORDS)) # not working

In [17]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
tf_idf_matrix = tfidf_vectorizer.fit_transform(refsDf['ProcessedRef'])
print(tf_idf_matrix[0])
refsDf['ProcessedRef'].iloc[0]

  (0, 74915)	0.21195646262231693
  (0, 121137)	0.2325964971431815
  (0, 13794)	0.26146462461044706
  (0, 34994)	0.28483105340733644
  (0, 118448)	0.2933245783787395
  (0, 139622)	0.3071069238095125
  (0, 184752)	0.30501228511578876
  (0, 58897)	0.3040295707600868
  (0, 82572)	0.1981463820488058
  (0, 74898)	0.1823588790524863
  (0, 121025)	0.17288865340932277
  (0, 13526)	0.1644314698595064
  (0, 34962)	0.17783397505396611
  (0, 118445)	0.24639235543357263
  (0, 139608)	0.21584841793765408
  (0, 184646)	0.19992128813685942
  (0, 58890)	0.2780688456810141


'computer-assisted system photographic mark-recapture analysis 2012 methods ecol. evol'

In [10]:
# refsDf['CitingTitle'] = refsDf['Title'].apply(lambda x: text_prepare(x, STOPWORDS))
# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
# tf_idf_matrix1 = tfidf_vectorizer.fit_transform(refsDf['CitingTitle'])

# print(tf_idf_matrix1[0])
# refsDf['CitingTitle'].iloc[0]

In [18]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [19]:
t1 = time.time()

# adjust lower bound: 0.8
# keep top 10 similar results
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)

t = time.time()-t1
print("finished in:", t)

finished in: 422.572092294693


In [13]:
# t1 = time.time()

# # adjust lower bound: 0.8
# # keep top 10 similar results
# matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix1, 10, 0.8)

# t = time.time()-t1
# print("finished in:", t)

In [20]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'ProcessedRef': left_side,
                          'SIMILAR_TITLE': right_side,
                           'similairity_score': similairity})

In [15]:
# need to think about how to get ID linked to this
# obvs ID for all refs should be NA or 0 
# Different col for citingID maybe?
# ID col for actual papers 
# or investigate how to do this for two seperate date sets

# def get_matches_df1(sparse_matrix, name_vector, top=100):
#     non_zeros = sparse_matrix.nonzero()
    
#     sparserows = non_zeros[0]
#     sparsecols = non_zeros[1]
    
#     if top:
#         nr_matches = top
#     else:
#         nr_matches = sparsecols.size
    
#     left_side = np.empty([nr_matches], dtype=object)
#     right_side = np.empty([nr_matches], dtype=object)
#     similairity = np.zeros(nr_matches)
#     ID = np.empty([nr_matches], dtype=object)
    
#     for index in range(0, nr_matches):
#         left_side[index] = name_vector[sparserows[index]]
#         right_side[index] = name_vector[sparsecols[index]]
#         similairity[index] = sparse_matrix.data[index]
#         ID[index] = 1
    
#     return pd.DataFrame({'ProcessedRef': left_side,
#                           'SIMILAR_TITLE': right_side,
#                            'similairity_score': similairity,
#                            'ID': ID,
#                            })

In [21]:
matches_df = pd.DataFrame()
matches_df = get_matches_df(matches, refsDf['ProcessedRef'], top=10000)
# Remove all exact matches
matches_df = matches_df[matches_df['similairity_score'] < 0.99999] 
matches_df.sample(10)
matches_df.sort_values(['similairity_score'], ascending=False).head(30)

,ProcessedRef,SIMILAR_TITLE,similairity_score
1871,i̇lemin gürkan b status activity patterns cara...,status activity patterns caracal caracal carac...,0.994155
1872,i̇lemin gürkan b status activity patterns cara...,status activity patterns caracal caracal carac...,0.994155
8900,drones count gulls? minimal disturbance semiau...,drones count gulls? minimal disturbance semiau...,0.992591
7448,wolves modulate soil nutrient heterogeneity fo...,wolves modulate soil nutrient heterogeneity fo...,0.991789
7447,wolves modulate soil nutrient heterogeneity fo...,wolves modulate soil nutrient heterogeneity fo...,0.991789
9277,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.991437
9276,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.991437
9275,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.991437
3387,three critical factors affecting automated ima...,three critical factors affecting automated ima...,0.990425
3386,three critical factors affecting automated ima...,three critical factors affecting automated ima...,0.990425


In [17]:
# maybe 
matches_df1 = get_matches_df(matches, refsDf['Title'], top=300000)
matches_df1 = matches_df[matches_df['similairity_score'] > 0.5] 
matches_df1.sample(10)
matches_df1.sort_values(['similairity_score'], ascending=False).head(30)

,ProcessedRef,SIMILAR_TITLE,similairity_score
1940,i̇lemin gürkan b status activity patterns cara...,status activity patterns caracal caracal carac...,0.995398
1939,i̇lemin gürkan b status activity patterns cara...,status activity patterns caracal caracal carac...,0.995398
9169,drones count gulls? minimal disturbance semiau...,drones count gulls? minimal disturbance semiau...,0.994427
7674,wolves modulate soil nutrient heterogeneity fo...,wolves modulate soil nutrient heterogeneity fo...,0.993995
7675,wolves modulate soil nutrient heterogeneity fo...,wolves modulate soil nutrient heterogeneity fo...,0.993995
9763,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.993516
9764,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.993516
9765,three novel methods estimate abundance unmarke...,three novel methods estimate abundance unmarke...,0.993516
174,remote infrared cameras used differentiate sma...,remote infrared cameras used differentiate sma...,0.992385
3617,remote infrared cameras used differentiate sma...,remote infrared cameras used differentiate sma...,0.992385


In [31]:
matches_df1 = get_matches_df(matches, refsDf['Title'], top=300000)
matches_df = get_matches_df(matches, refsDf['ProcessedRef'], top=351103)

In [18]:
# matches_df = pd.DataFrame()
# matches_df = get_matches_df(matches, refsDf['CitingTitle'], top=10000)
# # Remove all exact matches
# matches_df = matches_df[matches_df['similairity_score'] < 0.99999] 
# matches_df.sample(10)
# matches_df.sort_values(['similairity_score'], ascending=False).head(30)

In [19]:
#refsDf = pd.read_csv('citationNetworks/Data/SplitRefs.csv')
#print(refsDf.head())
#fuzz.ratio(refsDf["Title"], refsDf["Reference"])



In [2]:
def ngrams(string, n=3):
    string = ftfy.fix_text(string)
    string = string.encode("ascii", errors="ignore").decode()
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()
    string = re.sub(' +', ' ', string).strip()
    string = ' '+ string +' '
    string = re.sub(r'[,-./]\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]



In [3]:
def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

In [26]:
refsDf = pd.read_csv('citationNetworks/Data/SplitRefs.csv')
#refsDf['ref_title'] = str(refsDf['ref_title'])
refsDf = refsDf[['X','ID','Title', 'ref_title']]
refsDf.set_axis(['Index', 'ID','Title', 'ref_title'], axis=1)
# titlesAndIDs = refsDf[['ID','Title']]

#titlesAndIDs.set_axis(['ID', 'ref_title'], axis=1)
#refsDf = pd.concat([refsDf, titlesAndIDs])
#refsDf.reset_index(inplace=True, drop=True)

refsDf['ref_title'] = refsDf['ref_title'].astype(str)
# titlesAndIDs['Title'] = titlesAndIDs['Title'].astype(str)

cleanTitles = pd.read_csv('citationNetworks/Data/cleanTitles.csv')

In [27]:
# cleanTitles = refsDf[['ID','Title']]
cleanTitles['title'] = cleanTitles['title'].astype(str)
# cleanTitles = cleanTitles.iloc[:, 0:6]
cleanTitle = cleanTitles['title'].unique()

In [28]:
# titlesAndIDs = titlesAndIDs.iloc[:, 0:6]
# print(titlesAndIDs)
# titlesAndIDs = titlesAndIDs['Title'].unique()
# cleanTitles = titlesAndIDs['Title'].unique()
print('Vecorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(cleanTitle)
print('Vecorizing completed...')

Vecorizing the data - this could take a few minutes for large datasets...
Vecorizing completed...


In [29]:
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
citedTitle = 'ref_title' #column to match against in the messy data
uniqueCitedTitle = set(refsDf[citedTitle].values) # set used for increased performance###matching query:

In [30]:
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(uniqueCitedTitle)
t = time.time()-t1
print("COMPLETED IN:", t)

getting nearest n...


In [ ]:
uniqueCitedTitle = list(uniqueCitedTitle) #need to convert back to a list
print('finding matches...')
matches = []

for i,j in enumerate(indices):
  temp = [round(distances[i][0],2), cleanTitles.values[j][0][0], uniqueCitedTitle[i]]
  matches.append(temp)

finding matches...


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name'])
print('Done')

In [ ]:

# from sklearn.feature_extraction.text import TfidfVectorizer
# org_names = names['buyer'].unique()
# vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
# tf_idf_matrix = vectorizer.fit_transform(org_names)

# clean_org_names = pd.read_excel('Gov Orgs ONS.xlsx')
# clean_org_names = clean_org_names.iloc[:, 0:6]
# org_name_clean = clean_org_names['Institutions'].unique()


# print('Vecorizing the data - this could take a few minutes for large datasets...')
# vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
# tfidf = vectorizer.fit_transform(org_name_clean)
# print('Vecorizing completed...')


# nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
# org_column = 'buyer' #column to match against in the messy data
# unique_org = set(names[org_column].values) # set used for increased performance


# t1 = time.time()

# print('getting nearest n...')
# distances, indices = getNearestN(unique_org)

# t = time.time()-t1
# print("COMPLETED IN:", t)

# unique_org = list(unique_org) #need to convert back to a list
# print('finding matches...')
# matches = []

# for i,j in enumerate(indices):
#   temp = [round(distances[i][0],2), clean_org_names.values[j][0][0],unique_org[i]]
#   matches.append(temp)

# print('Building data frame...')  
# matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name'])
# print('Done')

The above basically works, however it is matching only within the single column (so just with titles of references). 
Potentially this can be solved by adding in the the citing papers titles to this column, with their actual IDs? 
Might be that from the Index column we can work back to getting the IDs linked back up. 

In [20]:
# reduce to unique 
# uniqueRefs = refsDf.drop_duplicates(subset=['ref_title'])

In [21]:
# for i in range(len(uniqueRefs)):
#     title = uniqueRefs.iloc[i,6] # 6 or title
#     refTitle = uniqueRefs.iloc[i,7] # 7 or ref title
#     ID = uniqueRefs.iloc[i,4] # 4 or ID
#     ratio = fuzz.ratio(title, refTitle)
#     if ratio > 50:
#         #append to a df with title, re_title, ID, ratio
#         tmpDf = pd.DataFrame([[ID, title, refTitle, ratio]], columns=list('ID', 'Title', 'RefTitle', 'PercentMatch'))




In [22]:
# for index, row in uniqueRefs.iterrows():
#     title = row['Title'] # 6 or title
#     refTitle = row['ref_title'] # 7 or ref title
#     ID = row['ID'] # 4 or ID
#     ratio = fuzz.ratio(title, refTitle)
#     # print(ratio)
#     # if ratio > 50:
#     #     #append to a df with title, re_title, ID, ratio
#     #     tmpDf = pd.DataFrame([[ID, title, refTitle, ratio]], columns=list('ID', 'Title', 'RefTitle', 'PercentMatch'))
    

In [23]:
# nan_value = float("NaN")
# uniqueRefs = uniqueRefs.replace("", nan_value)
# uniqueRefs = uniqueRefs.dropna(subset=["ref_title"])

In [24]:
# !jupyter nbextension enable --py widgetsnbextension

In [25]:
# miniUniqueRefs = uniqueRefs.iloc[0:10]
# titleDf = uniqueRefs = refsDf.drop_duplicates(subset=['Title'])#
# #tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
# tmpDf = pd.DataFrame(columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID'))
# columns = list(tmpDf)
# newDF = []

# for index, row in tqdm(miniUniqueRefs.iterrows()):
#     title = row['Title'] # 6 or title
#     refTitle = str(row['ref_title']) # 7 or ref title
#     ID = row['ID'] # 4 or ID
#     output = process.extractOne(refTitle, titleDf['Title'], scorer=fuzz.token_sort_ratio)
#     # print(output)
#     ratio = output[1]
#     bestMatch = output[0]
#     bestMatchID = output[2]
#     values = [ID, title, refTitle, ratio, bestMatch, bestMatchID]
#     zipped = zip(columns, values)
#     dictionary = dict(zipped)
#     newDF.append(dictionary)
#     # tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
#     # tmpDf.columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID')
#     # tmpDf.append(tmpDf)
# print(tmpDf)

In [26]:
# titleDf = uniqueRefs = refsDf.drop_duplicates(subset=['Title'])#
# #tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
# tmpDf = pd.DataFrame(columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID'))
# columns = list(tmpDf)
# newDF = []

# #with alive_bar(238148, force_tty=True) as bar:
# for index, row in tqdm(uniqueRefs.iterrows()):
#     title = row['Title'] # 6 or title
#     refTitle = str(row['ref_title']) # 7 or ref title
#     ID = row['ID'] # 4 or ID
#     output = process.extractOne(refTitle, titleDf['Title'], scorer=fuzz.token_sort_ratio)
#     # print(output)
#     ratio = output[1]
#     bestMatch = output[0]
#     bestMatchID = output[2]
#     values = [ID, title, refTitle, ratio, bestMatch, bestMatchID]
#     zipped = zip(columns, values)
#     dictionary = dict(zipped)
#     newDF.append(dictionary)
#     #tmpDf = pd.DataFrame([[ID, title, refTitle, ratio, bestMatch, bestMatchID]]) # add bits needed
#     #tmpDf.columns=('ID', 'Title', 'RefTitle', 'PercentMatch', 'BestMatchTitle', 'BestMatchID')
#     #tmpDf.append(tmpDf)
# #bar()
    

In [27]:
# miniUniqueRefs = uniqueRefs.iloc[0:10]
# df0_names = list(miniUniqueRefs.ref_title.unique())
# df1_names = list(uniqueRefs.ref_title.unique())
# df2_names = list(uniqueRefs.Title.unique())
# print(process.extractOne(str(df0_names), df2_names, scorer=fuzz.token_sort_ratio))
# #print(process.extractOne(str(df1_names), df2_names, scorer=fuzz.token_sort_ratio))


In [28]:
# def match_names(name, list_names, min_score=0):
#     max_score = -1
#     max_name = ''
#     for x in list_names:
#         score = fuzz.ratio(name, x)
#         if (score > min_score) & (score > max_score):
#             max_name = x
#             max_score = score
#     return (max_name, max_score)

In [29]:
# df1_names = list(uniqueRefs.ref_title.unique())
# titlesList = list(uniqueRefs.Title.unique())
# refsAsArray = uniqueRefs.to_numpy()
# miniUniqueRefs = uniqueRefs.iloc[0:10]
# miniArray = miniUniqueRefs.to_numpy()

# print(process.extractOne(str(miniArray[:,5]), titlesList, scorer=fuzz.token_sort_ratio))
